In [1]:
import tensorflow as tf

In [8]:
layer = tf.keras.layers.Dense(10, input_shape=(None, 5))

layer(tf.zeros([10, 5]))

<tf.Tensor: shape=(10, 10), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>

In [9]:
print(layer.variables)
print(layer.kernel)
print(layer.bias)

[<tf.Variable 'dense_3/kernel:0' shape=(5, 10) dtype=float32, numpy=
array([[ 0.580509  ,  0.10680091,  0.40022153, -0.5309654 , -0.29894567,
         0.23516273,  0.22106683, -0.45090947,  0.4494961 ,  0.531576  ],
       [ 0.40484446, -0.05868608,  0.1180557 , -0.30863056,  0.08916843,
        -0.20249596,  0.559524  ,  0.14689142, -0.47402304,  0.12226468],
       [-0.28474963,  0.2379083 , -0.5605786 , -0.36421144,  0.13568568,
         0.0782454 ,  0.11518693, -0.58305955,  0.21440226, -0.41624773],
       [-0.34198812, -0.21451926, -0.61825377,  0.56118137,  0.04873943,
         0.00723064,  0.25954175,  0.00711757, -0.31319946,  0.44594508],
       [-0.32208428,  0.02578253,  0.12774885,  0.42982715, -0.14622793,
         0.30597305,  0.34208888, -0.07750034, -0.58661485,  0.06908888]],
      dtype=float32)>, <tf.Variable 'dense_3/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>]
<tf.Variable 'dense_3/kernel:0' shape=(5, 10

### Custom Layer

In [10]:
class Linear(tf.keras.layers.Layer):
    def __init__(self, units=32, input_dim=32):
        super(Linear, self).__init__()
        
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(initial_value=w_init(shape=(input_dim, units),
                                                 dtype='float32'),
                            trainable=True)
        
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(initial_value=b_init(shape=(units,),
                                                 dtype='float32'),
                            trainable=True)
        
    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

In [12]:
x = tf.ones((2, 2))
print(f'x: {x}')

linear = Linear(4, 2)
y = linear(inputs)
print(f'y: {y}')

x: [[1. 1.]
 [1. 1.]]
y: [[ 0.06697868 -0.01135617  0.0569241   0.03401322]
 [ 0.06697868 -0.01135617  0.0569241   0.03401322]]


In [17]:
assert linear.weights == [linear.w, linear.b]

### Custom Layer with self.add_weight()

In [18]:
class Linear(tf.keras.layers.Layer):
    def __init__(self, units=32, input_dim=32):
        super(Linear, self).__init__()
        self.w = self.add_weight(shape=(input_dim, units),
                                initializer='random_normal',
                                trainable=True)
        self.b = self.add_weight(shape=(units,),
                                initializer='zeros',
                                trainable=True)
    
    def call(self, x):
        return tf.matmul(x, self.w) + self.b

In [19]:
x = tf.ones((2, 2))
print(f'x: {x}')

linear = Linear(4, 2)
y = linear(inputs)
print(f'y: {y}')

x: [[1. 1.]
 [1. 1.]]
y: [[-0.01044869 -0.02761954  0.09738257 -0.06461011]
 [-0.01044869 -0.02761954  0.09738257 -0.06461011]]


### Custom Layer with build()

In [25]:
# In many cases, you may not know in advance the size of your inputs, and 
# you would like to lazily create weights when that value becomes known, 
# some time after instantiating the layer.
class Linear(tf.keras.layers.Layer):
    def __init__(self, units=32):
        super(Linear, self).__init__()
        self.units = units
        
    def build(self, input_shape):
        self.w = self.add_weight(shape=(input_shape[-1], self.units),
                                initializer='random_normal',
                                trainable=True)
        self.b = self.add_weight(shape=(self.units,),
                                initializer='zeros',
                                trainable=True)
        
    def call(self, x):
        return tf.matmul(x, self.w) + self.b

In [26]:
x = tf.ones((2, 2))
print(f'x: {x}')

linear = Linear(4)
y = linear(inputs)
print(f'y: {y}')

x: [[1. 1.]
 [1. 1.]]
y: [[-0.00039262 -0.04449359  0.04367013  0.09235971]
 [-0.00039262 -0.04449359  0.04367013  0.09235971]]
